In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
dados =pd.read_csv('databasePJF.csv', header=0, sep=',', quotechar='"')

In [5]:
dados

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.00,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.00,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.00,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.00,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.00,40348.79,0,0


In [31]:
#criando um dataframe com menos não fraudes para tornar maior a proporção de fraudes para não fraudes.
dadosfraud = dados[(dados.isFraud == 1)]
dadosnfraud = dados[(dados.isFraud == 0)]
dadosnfraud = dadosnfraud.ix[0:25000]
dadosfinais = pd.concat([dadosfraud,dadosnfraud])
dadosfinais = dadosfinais.sample(frac = 1)
dadosfinais = dadosfinais.reset_index()
dadosfinais

,index,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,13578,7,PAYMENT,2481.93,C356545825,14910.00,12428.07,M1460130079,0.00,0.00,0,0
1,1030510,69,CASH_OUT,175328.85,C215072412,175328.85,0.00,C386798101,110129.95,285458.80,1,0
2,15468,8,PAYMENT,13375.88,C706838997,0.00,0.00,M317858117,0.00,0.00,0,0
3,2324509,188,CASH_OUT,237642.64,C84594080,237642.64,0.00,C855526143,113161.91,350804.55,1,0
4,11128,7,PAYMENT,12291.98,C1074820946,40034.48,27742.50,M472933779,0.00,0.00,0,0
5,11266,7,TRANSFER,322464.50,C1483430324,25475.00,0.00,C344247127,125807.00,0.00,0,0
6,7141,6,PAYMENT,336.05,C1098082328,0.00,0.00,M1165836116,0.00,0.00,0,0
7,21232,8,PAYMENT,14079.66,C1104169600,0.00,0.00,M1134229208,0.00,0.00,0,0
8,24115,8,TRANSFER,610493.06,C1951112189,201747.00,0.00,C1115240385,374895.28,1008238.91,0,0
9,24816,8,PAYMENT,7585.83,C1175274169,15679.35,8093.52,M1118771427,0.00,0.00,0,0


In [32]:
dadosfinais.columns

Index(['index', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
       'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [40]:
#utilizando o naive bayes retirado do site https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
#utilizando a mesma base de dados utilizada para treinar o naive bayes.
y = dadosfinais.isFraud
x = dadosfinais.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x,y)
predicted = model.predict(x)
print(predicted)
#help(model.predict)

[0 0 0 ..., 1 0 1]


In [48]:
#comparando o real com a classficação do naive bayes
x.loc[:, ["isFraud", "predicted"]]

,isFraud,predicted
0,0,0
1,1,0
2,0,0
3,1,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [49]:
x["isFraud"] = y

In [ ]:
#para melhrar: dar shuffle antes de pegar os 25 mil não fraudes, calcular a diferença entre o naive bayes e o real, e criar um base de testes.